# Neural Style


[Neural style transfer](https://arxiv.org/abs/1508.06576) 就是使用神经网络来完成将一张指定图片的风格应用到另外一张图片上的算法。


![neural-style](./neural-style.svg)



它的流程如下所示：


![neural-style2](./neural-style2.svg)



文字部分的描述就是：


1、准备好内容图片 $content$ 和样式图片$style$。

2、初始化合成图片 $x$ ，然后把 $x$ 、$content$ 、$style$ 都扔进 VGG19提取特征，其中的 $x$ 和 $style$ 的层1, 2, 4来对样式层进行匹配； $x$ 和 $content$ 的层3作为内容层来匹配。

3、使用 $style \_ Loss$ 和 $content \_ Loss$ 来分别计算样式损失和内容损失，并对输入 $x$ 求导，导数记作 $g$

4、更新输入 $x$ 的值 $x = x - lr * g$




说明部分图片来源：https://zh.gluon.ai/chapter_computer-vision/neural-style.html

In [109]:
import torch
from torch.autograd import Variable
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import torchvision

In [108]:
class Config(object):
    IMAGENET_MEAN = [0.485, 0.456, 0.406] # IMAGENET中的图片归一化
    IMAGENET_STD = [0.229, 0.224,  0.225] # IMAGENET中的图片归一化
    imsize = 512 # 统一 content image 和 style image 的 size
    style_image_path = "./images/picasso.jpg"
    content_image_path = "./images/dancing.jpg"
    DOWNLOAD = False # 是否下载预训练模型
    lr = 1e-3 # 学习速率
    epoches = 500 # 训练epoch
    sample_epoch = 50 # 显示损失的epoch
    c_weight = 1e5 # content_loss 的权重 
    s_weight = 1e10 # style_loss的权重
# 参数配置函数
config = Config()

In [49]:
# 定义预处理函数
# 用于把原始图片进行归一化，转换成卷积神经网络可以接收的输入格式
def preprocess(image_path, trasform=None):
    image = Image.open(image_path)
    image = Variable(trasform(image))
    image = image.unsqueeze(0)
    return image

transformer = transforms.Compose([
        transforms.Scale(config.imsize),
        transforms.ToTensor()
        transforms.Normalize(mean = config.IMAGENET_MEAN,std = config.IMAGENET_STD)
    ])

pltshow = transforms.ToPILImage()

# 图片展示
def imshow(tensor, title=None):
    image = tensor.clone().cpu()
    image = image.view(3, config.imsize, config.imsize)
    image = pltshow(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

In [103]:
# 定义模型
class VGG_extract(nn.Module):
    def __init__(self):
        super(VGG_extract, self).__init__()
        self.style_layers = [0,5,10,19,28]
        self.content_layers = [25]
        self.vgg = models.vgg19(pretrained=config.DOWNLOAD).features
    
    def forward(self, x):
        # 由于我们只需要指定层的输出，所以，我们同时构建特征提取函数只保留制定层的值
        contents = []
        styles = []  
        for i in range(len(net)):
            x = net[i](x)
            if i in style_layers:
                styles.append(x)
            if i in content_layers:
                contents.append(x)
        return contents, styles


In [87]:
# 构建损失函数 （最麻烦的地方了...）

# 内容匹配只涉及一层，所以可以看成回归问题，直接使用均方误差。
def content_loss(y_hat, y):
    return torch.mean( (y_hat - y) ** 2 )

# 样式匹配则是通过拟合Gram矩阵
def gram(y):
    b, c, h, w = y.size()
    y = y.view(c, h * w)
    norm = c * h * w
    return torch.mm(y, y.t()) / norm

# 计算样式损失
def style_loss(yhat, y):
    return torch.mean((gram(yhat) - gram(y)) ** 2)

In [91]:
# 开始训练
style = preprocess(config.style_image_path, transformer)
content = preprocess(config.content_image_path, transformer)

target = Variable(content.clone(), requires_grad=True) # 这就是我们需要更新的是输入 x
optimizer = torch.optim.Adam([target], lr=config.lr, betas=[0.5, 0.999])

# 加载模型
vgg = VGG_extract()

if torch.cuda.is_available():
    vgg = vgg.cuda()
    
for epcho in range(config.epoches):
    # 获取特征
    t_c_fea, t_s_fea = vgg(target)
    c_fea, _ = vgg(content)
    _, s_fea = vgg(style)
    
    # 计算损失
    c_loss = content_loss(t_c_fea, c_fea)
    s_loss = style_loss(t_s_fea, s_fea)
    loss = config.c_weight * c_loss + config.s_weight * s_loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epcho+1) % config.show_epoch == 0:
        print ('Epcho [%d/%d], Content Loss: %.4f, Style Loss: %.4f' 
               %(epcho+1, config.epoches, c_loss, s_loss))
        # 保存图片
        denorm = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
        img = target.clone().cpu().squeeze()
        img = denorm(img.data).clamp_(0, 1)
        torchvision.utils.save_image(img, 'output-%d.png' %(epcho+1))